<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/logo-bdc.png" align="right" width="64"/>

# <span style="color:#336699">3º BIG TechTalks - Acesso, Visualização e Processamento de Imagens Sentinel-2 utilizando Python - Web Time Series Service (WTSS)</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Rennan F. B. Marujo e Gilberto R. Queiroz
    <br/><br/>
    Divisão de Observação da Terra e Geoinformática, Instituto Nacional de Pesquisas Espaciais (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contato: <a href="mailto:rennan.marujo@inpe.br">rennan.marujo@inpe.br</a>
    <br/><br/>
    Ultíma Atualização: 24 de Abril de 2025
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Resumo.</b> Este Jupyter Notebook é parte do 3º BIG TechTalks - Acesso, Visualização e Processamento de Imagens Sentinel-2 utilizando Python. Este Jupyter Notebook apresenta uma visão geral de como utilizar o serviço Web Time Series Service (WTSS) na linguagem Python para extrair séries temporais de produtos de dados de sensoriamento remoto disponíveis no catálogo do INPE.
</div>

## <span style="color:#336699">Introdução ao Web Times Series Service (WTSS)<hr style="border:1px solid #0077b9;">

O **W**eb **T**ime **S**eries **S**ervice ou **WTSS** é um serviço web voltado para extração de séries temporais a partir de imagens de sensoriamento remoto organizadas na forma de cubos de dados. Dada uma localização ou região do espaço geográfico e um intervalo de tempo, a API deste serviço permite que os clientes requisitem as séries temporais para os produtos de dados do [Brazil Data Cube](https://data.inpe.br/bdc/web/) (Figura 1). Esse serviço pode ser utilizado a partir de um pacote construído para linguagem de programação [Python](https://github.com/brazil-data-cube/wtss.py), ou pode ser usado diretamente em interfaces gráficas com o usuário, por meio de [plugin do QGIS](https://github.com/brazil-data-cube/wtss-qgis) ou de componente gráfico de aplicações como o [Data Cube Explorer](https://data.inpe.br/bdc/explorer/) e [TerraCollect](https://data.inpe.br/bdc/terracollect/).

<center>
    <img src="https://geo-credito-rural.github.io/_images/overview.png" width="80%" />
    <br/>
    <b>Figura 1</b> - Visão geral do serviço WTSS.
</center>


<br/><br/>


O WTSS é baseado em três operações (Figura 2):
- `list_coverages`: Retorna a lista de produtos de dados, isto é, cubos de dados, disponíveis no serviço para consulta.

- `describe_coverage`: Retorna metadados básicos de um determinado cubo de dados.

- `time_series`: Consulta a lista de valores associadas a uma dada localização ou região do espaço geográfico e um intervalo de tempo.


**Nota:** O projeto original do WTSS utilizava a nomenclatura de *coverage* para se referir aos produtos de dados regulares no espaço e no tempo, como é o caso dos cubos de dados do Brazil Data Cube.


<br/><br/>


<center>    
    <img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/wtss/wtss-operations.png?raw=true" align="center" width="768"/>
    <br/>
    <b>Figura 2</b> - Operações do Serviço WTSS.
    <br/>
    <b>Fonte</b>: Galeria de Código do Brazil Data Cube.
</center>


<br/><br/>


O pacote para a linguagem Python fornece abstrações próprias para utilização dessas operações.

## Cliente WTSS no Python
<hr style="border:1px solid #0077b9;">

<br/>

Para demonstrar o acesso aos produtos de dados do Brazil Data Cube, iremos utilizar uma bibloteca de software livre para Python denominada [wtss.py](https://github.com/brazil-data-cube/wtss.py) (`wtss`).

Para instalar essa biblioteca no ambiente Jupyter, pode ser utilizado o seguinte comando `pip install`:

In [ ]:
!pip install wtss==2.0.0a3

Uma vez instalada a biblioteca `wtss`, podemos carregar suas funcionalidades através do comando `import`, como mostrado abaixo:

In [ ]:
import wtss

Em geral, uma biblioteca do ecossistema Python possui uma constante especial para informar a versão da biblioteca carregada. Abaixo, apresentamos a versão carregada  da biblioteca `wtss`:

In [ ]:
wtss.__version__

<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/wtss/list-coverages.png?raw=true" align="right" width="220"/>

## Verificando os Produtos de Dados Disponíveis no Serviço
<hr style="border:1px solid #0077b9;">

<br/><br/>

O endereço do serviço WTSS do BDC é https://data.inpe.br/bdc/wtss/v4/. Para descobrir os cubos de dados disponíveis para acesso nesse serviço,
é possível utilizar a classe `WTSS` do pacote `wtss`. Ao criar um objeto dessa classe, devemos informar a URL do serviço:

In [ ]:
servico = wtss.WTSS('https://data.inpe.br/bdc/wtss/v4/')

O WTSS utiliza a terminologia de **coverages** para se referir aos cubos de dados do Brazil Data Cube. A propriedade `coverages` retorna uma lista com os identificadores dos cubos de dados, isto é, das *coverages*, que podem ser consultadas pelo serviço:

In [ ]:
servico.coverages

Repare que os identificadores retornados pela propriedade `coverages` são os mesmos usados no catálogo STAC. Seus significados podem ser consultados no [Data Cube Explorer](https://data.inpe.br/bdc/explorer/), ou no [STAC Browser](https://data.inpe.br/stac/browser/) ou diretamente no [serviço STAC](https://data.inpe.br/bdc/stac/).

**Observação:** A documentação da classe `WTSS` pode ser vista [aqui](https://wtss.readthedocs.io/en/latest/class_wtss.html).

<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/wtss/describe-coverage.png?raw=true" align="right" width="220"/>

## Recuperando os Metadados de um Produto
<hr style="border:1px solid #0077b9;">

<br/>

Os identificadores retornados pela propriedade `coverages` podem ser utilizados nas operações seguintes do serviço, tanto para recuperação de metadados quanto da série temporal. O operador `[]` em um objeto `WTSS` permite acessar o metadado de um cubo de dados específico. Por exemplo, vamos considerar o cubo de dados Sentinel-2, com resolução espacial de 10 metros e composição temporal de 16 dias. Este produto é identificado na lista anterior como `S2-16D-2`. Vamos recuperar seus metadados no serviço WTSS:

In [ ]:
s2 = servico['S2-16D-2']
s2

Os metadados acima são um subconjunto dos metadados disponíveis no STAC e incluem:

- nome do cubo de dados

In [ ]:
s2.name

- descrição:

In [ ]:
s2.description

- Atributos (bandas ou índices espectrais):

In [ ]:
s2.attributes

- Retângulo envolvente da base de imagens do cubo de dados, representado como uma geometria do tipo `Polygon` da bibloteca `Shapely`.

In [ ]:
s2.spatial_extent

- Instantes de tempo das obervações disponíveis no cubo.

In [ ]:
timeline = s2.timeline

ultima_data_disponivel = timeline[0]
primeira_data_disponivel = timeline[-1]

print(f'Intervalo: [{primeira_data_disponivel}, {ultima_data_disponivel}]')

**Observação:** A documentação da classe `Coverage` pode ser vista [aqui](https://wtss.readthedocs.io/en/latest/class_coverage.html).

<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/wtss/time-series.png?raw=true" align="right" width="220"/>

## Recuperando a Série Temporal
<hr style="border:1px solid #0077b9;">

<br/><br/>

O método `ts` de um objeto `Coverage` permite recuperar a série temporal para uma dada localização ou região e conjunto de atributos. Vamos considerar o `cubo_s2` obtido anteriormente e recuperar as séries temporais para os atributos `NDVI` e `B08` (banda do infravermelho próximo ou NIR) na localização de latitude `-12.32800` e longitude `-45.90710` entre 01 de Janeiro de 2022 e 31 de Dezembro de 2023:

In [ ]:
lat, lon = -12.32800, -45.90710

In [ ]:
import folium

f = folium.Figure(width=1000, height=300)

m = folium.Map(location=[lat, lon],
               zoom_start=11,
               tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
               attr="Esri World Imagery"
).add_to(f)

folium.Marker(
    [lat, lon],
    popup="Pto",
).add_to(m)

m

In [ ]:
ts_search = s2.ts(attributes=('NDVI', 'B08'),
                  latitude=lat,
                  longitude=lon,
                  start_date='2022-01-01',
                  end_date='2025-03-21')
ts_search

A propriedade `ts` permite recuperar um objeto representando as séries temporais:

In [ ]:
ts = ts_search.ts
ts

O objeto retornado `ts_search` (`TimeSeriesSearch`) possui métodos para recuperação da série temporal e para sua visualização:

In [ ]:
ts_search.plot()

O eixo do tempo, com os valores de datas, pode ser acessado com a propriedade `timeline`:

In [ ]:
ts.timeline

Os nomes dos atributos recuperados nas séries pode ser acessados pela propriedade `attributes`:

In [ ]:
ts.attributes

E os valores da série para cada atributo podem ser recuperados pelo método `values`:

In [ ]:
ts.values('NDVI')[0]

## Filtragem ou Remoção de Ruídos
<hr style="border:1px solid #0077b9;">

<br/><br/>

A biblioteca `scipy` se baseia na biblioteca `NumPy` e fornece funções adicionais para várias operações matemáticas e científicas, incluindo processamento de sinais e filtragem, dentre outras. Abaixo, são listados alguns módulos e funcionalidades que o SciPy oferece:

- Álgebra Linear (scipy.linalg).

- Otimização (scipy.optimize).

- Integração e EDOs (scipy.integrate).

- Interpolação (scipy.interpolate).

- Transformadas (scipy.fft).

- Estatísticas (scipy.stats).

- Processamento de Sinais (scipy.signal): Funções para processamento de sinais, incluindo filtros e análise de sistemas. Na filtragem abaixo iremos utilizar o módulo `scipy.signal` para aplicar o filtro **Savitzky–Golay** sobre as séries temporais obtidas.

In [ ]:
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
window_size = 10

poly_order = 5

ndvi_smooth = savgol_filter(ts.values('NDVI'), window_size, poly_order)
ndvi_smooth.T.shape

In [ ]:
np.linspace(0, len(ts.timeline), num=23, endpoint=True)

In [ ]:
fig = plt.figure( figsize=plt.figaspect(0.3) )

plt.xlabel('Data')
plt.ylabel('NDVI')

plt.xticks(np.linspace(0, len(ts.timeline) - 1, num=23, endpoint=True))

plt.plot(ts.timeline, ts.values('NDVI')[0], color="blue", linewidth=1.0, label='NDVI')

plt.plot(ts.timeline, ndvi_smooth.T, color="red", linewidth=1.0, label='NDVI Suavizado')

plt.grid(visible=True, color='gray', linestyle='--', linewidth=0.5)

plt.legend()

fig.autofmt_xdate()

plt.show()

## Recuperando a Série Temporal de uma área
<hr style="border:1px solid #0077b9;">

<br/><br/>

Nesta seção vamos apresentar as séries temporais associadas à área inteira.
primeiramente, vamos selecionar um unico poligono do nosso .shp inicial e visualiza-lo:

In [ ]:
import geopandas as gpd
import io
import os
import requests
import shapely
import tempfile
import warnings
import zipfile

zipfile_url = "https://github.com/brazil-data-cube/code-gallery/raw/master/jupyter/Data/2025-sbsr/LEM_dataset_small.zip"
response = requests.get(zipfile_url)
with tempfile.TemporaryDirectory() as tmpdir:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(tmpdir)

        shp_file = [f for f in os.listdir(tmpdir) if f.endswith('.shp')][0]
        shp_path = os.path.join(tmpdir, shp_file)

        my_shp = gpd.read_file(shp_path)

        warnings.filterwarnings("ignore")
        geometry_union = my_shp.geometry.union_all()
        bbox = geometry_union.bounds
        centroide = geometry_union.centroid

polygon = my_shp.iloc[22]
geometry = polygon.geometry
centroide = geometry.centroid
warnings.filterwarnings("default")

In [ ]:
m = folium.Map(
    location=[centroide.y, centroide.x],
    zoom_start=13,
)

folium.GeoJson(geometry).add_to(m)

m

Finalmente, vamos recuperar as séries temporais da area apresentada no mapa acima, por meio do client Python do serviço WTSS:

In [ ]:
ts_search = s2.ts(attributes=('NDVI', 'B08'),
                       geom=geometry,
                       start_date='2024-01-01', end_date='2025-03-21')
ts_search

In [ ]:
ts_search.plot()